In [1]:
# imports
import numpy as np
import pandas as pd

from tqdm import tqdm
import multiprocessing as mp

import lyricsgenius as lg

In [2]:
print("Number of processors: ", mp.cpu_count())

Number of processors:  8


In [3]:
genius=lg.Genius(access_token='Insert access_token',
                 skip_non_songs=True, remove_section_headers=True,sleep_time=0.1,verbose=False)

# Genre Selection

In [4]:
df = pd.read_csv('../data/track_audio_dataset_no_lyrics.csv', index_col=0)

In [5]:
df.genre.unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop', 'pop-film',
       'pow

In [6]:
# Create df of selected genres to sample lyrics from
mask = (df['genre'] == 'alternative') | (df['genre'] == 'country') | (df['genre'] == 'edm') | (df['genre'] == 'hip-hop') | (df['genre'] == 'r-n-b') | (df['genre'] == 'rock')
df_subset = df[mask]
# Gather track and artists
tracks = df_subset.track_name.copy().to_list()
artists = df_subset.artist_name.copy().to_list()

In [7]:
len(df_subset)

3274

In [8]:
def lyric_generator(track_lst,artist_lst):
    """
    Takes two ordered, same-size lists of tracks and artists and
    returns a list of each track's lyrics.
    
    Parameters:
    -----------
    track_lst: list of song titles
    artist_lst: list of artist names
    
    Output:
    -------
    Returns a list of lyrics (strings) for each song passed in.
    """
    lyric_lst = []
    # Initialize API object
    genius=lg.Genius(access_token='Insert access_token',
                     skip_non_songs=True, 
                     remove_section_headers=True,
                     sleep_time=0.1,
                     verbose=False)
    
    # Iterate through tracks and store lyrics
    for t, a in tqdm(zip(track_lst, artist_lst)):
        song = genius.search_song(title = t,
                                  artist = a,
                                  get_full_info = False)
        if song is not None:
            lyric_lst.append(song.lyrics)
        else:
            lyric_lst.append('')
        
    return lyric_lst

In [17]:
def single_song_lyric_generator(genius_API, track, artist):
    """
    Takes a track and its corresponding artist and
    returns the song's lyrics using the provided
    genius API object. If there aren't any lyrics,
    it returns a blank string.
    
    Parameters:
    -----------
    genius_API: genius API object
    track: song title
    artist: artist name
    
    Output:
    -------
    Returns the lyrics (string) for the specified song.
    """
    # Pull song from API
    song = genius_API.search_song(title = track,
                                  artist = artist,
                                  get_full_info = False)
    if song is not None:
        return song.lyrics
    return ''

# Testing parallel and non-parallel methods and comparing results

In [12]:
# Initialize API object
genius=lg.Genius(access_token='Insert access_token',
                 skip_non_songs=True,
                 remove_section_headers=True,
                 sleep_time=0.1,
                 verbose=False)

# Initialize multiprocessing object to make multiple API calls at once
pool = mp.Pool(mp.cpu_count())

test_parallel = [pool.apply(single_song_lyric_generator, args=(genius,t,a)) for t, a in tqdm(zip(tracks[:5],artists[:5]))]

# Close object
pool.close()


0it [00:00, ?it/s]
1it [00:01,  1.02s/it]
2it [00:02,  1.04s/it]
3it [00:03,  1.02s/it]
4it [00:07,  2.00s/it]
5it [00:08,  1.70s/it]


In [11]:
test = lyric_generator(tracks[:5],artists[:5])


0it [00:00, ?it/s]
1it [00:00,  1.16it/s]
2it [00:01,  1.20it/s]
3it [00:02,  1.24it/s]
4it [00:03,  1.23it/s]
5it [00:03,  1.26it/s]


In [13]:
test == test_parallel

True

# Final Lyric Pull

In [15]:
# Initialize multiprocessing object to make multiple API calls at once
pool = mp.Pool(mp.cpu_count())

lyrics = [pool.apply(single_song_lyric_generator, args=(genius,t,a)) for t, a in tqdm(zip(tracks,artists))]

# Close object
pool.close()



0it [00:00, ?it/s]

1it [00:00,  1.09it/s]

2it [00:02,  1.03it/s]

3it [00:03,  1.01it/s]

4it [00:04,  1.01it/s]

5it [00:05,  1.01s/it]

6it [00:07,  1.33s/it]

7it [00:10,  2.00s/it]

8it [00:12,  2.03s/it]

9it [00:14,  1.99s/it]

10it [00:16,  1.94s/it]

11it [00:18,  1.89s/it]

12it [00:20,  1.96s/it]

13it [00:22,  2.11s/it]

14it [00:24,  2.10s/it]

15it [00:27,  2.08s/it]

16it [00:29,  2.11s/it]

17it [00:31,  2.12s/it]

18it [00:33,  2.07s/it]

19it [00:35,  2.01s/it]

20it [00:37,  2.05s/it]

21it [00:39,  1.99s/it]

22it [00:41,  2.12s/it]

23it [00:43,  2.03s/it]

24it [00:45,  1.92s/it]

25it [00:47,  1.96s/it]

26it [00:49,  2.04s/it]

27it [00:51,  2.04s/it]

28it [00:53,  1.97s/it]

29it [00:58,  2.86s/it]

30it [01:00,  2.61s/it]

31it [01:03,  2.72s/it]

32it [01:05,  2.48s/it]

33it [01:08,  2.69s/it]

34it [01:10,  2.59s/it]

35it [01:12,  2.38s/it]

36it [01:14,  2.26s/it]

37it [01:16,  2.29s/it]

38it [01:19,  2.45s/it]

39it [01:22,  2.43s/it]

40it [01:23,

259it [09:43,  1.97s/it]

260it [09:45,  1.88s/it]

261it [09:47,  1.83s/it]

262it [09:50,  2.11s/it]

263it [09:52,  2.14s/it]

264it [09:55,  2.36s/it]

265it [09:56,  2.18s/it]

266it [09:58,  2.13s/it]

267it [10:01,  2.17s/it]

268it [10:03,  2.06s/it]

269it [10:06,  2.33s/it]

270it [10:09,  2.64s/it]

271it [10:11,  2.39s/it]

272it [10:14,  2.66s/it]

273it [10:16,  2.40s/it]

274it [10:18,  2.22s/it]

275it [10:21,  2.46s/it]

276it [10:23,  2.36s/it]

277it [10:24,  2.09s/it]

278it [10:27,  2.17s/it]

279it [10:29,  2.21s/it]

280it [10:31,  2.06s/it]

281it [10:33,  2.08s/it]

282it [10:35,  2.13s/it]

283it [10:38,  2.33s/it]

284it [10:40,  2.46s/it]

285it [10:43,  2.35s/it]

286it [10:45,  2.37s/it]

287it [10:48,  2.54s/it]

288it [10:50,  2.33s/it]

289it [10:52,  2.15s/it]

290it [10:53,  2.00s/it]

291it [10:56,  2.17s/it]

292it [10:57,  2.05s/it]

293it [10:59,  1.97s/it]

294it [11:01,  1.96s/it]

295it [11:03,  1.85s/it]

296it [11:05,  2.03s/it]

297it [11:07

515it [19:56,  2.08s/it]

516it [19:58,  2.04s/it]

517it [20:00,  2.03s/it]

518it [20:03,  2.42s/it]

519it [20:04,  1.91s/it]

520it [20:06,  2.13s/it]

521it [20:09,  2.20s/it]

522it [20:12,  2.40s/it]

523it [20:14,  2.46s/it]

524it [20:16,  2.27s/it]

525it [20:19,  2.44s/it]

526it [20:21,  2.38s/it]

527it [20:24,  2.47s/it]

528it [20:26,  2.50s/it]

529it [20:29,  2.50s/it]

530it [20:31,  2.26s/it]

531it [20:32,  1.94s/it]

532it [20:33,  1.82s/it]

533it [20:35,  1.92s/it]

534it [20:37,  1.87s/it]

535it [20:39,  1.83s/it]

536it [20:41,  1.92s/it]

537it [20:43,  1.87s/it]

538it [20:46,  2.14s/it]

539it [20:48,  2.27s/it]

540it [20:50,  2.05s/it]

541it [20:51,  1.98s/it]

542it [20:53,  1.89s/it]

543it [20:55,  1.85s/it]

544it [20:58,  2.15s/it]

545it [21:01,  2.39s/it]

546it [21:03,  2.40s/it]

547it [21:05,  2.28s/it]

548it [21:07,  2.21s/it]

549it [21:09,  2.03s/it]

550it [21:10,  1.88s/it]

551it [21:12,  1.85s/it]

552it [21:14,  1.82s/it]

553it [21:16

Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)